## << SLM Fine-tuning 실습: LoRA를 활용한 나만의 챗봇 만들기 >>

- 본 실습은 Google Colab T4 GPU 환경에서 **LoRA**를 활용하여 SLM을 파인튜닝합니다.
- 모든 코드는 **Google Colab T4 GPU** 환경을 기준으로 작성되었습니다.

**실습 목표**

**1. 환경 설정 및 기본 추론 (Environment Setup & Inference)**
- 필수 라이브러리를 설치하고 GPU 환경을 확인합니다.
- 베이스 모델(Qwen3-4B-Instruct)을 로드하여 Fine-tuning 전 성능을 테스트합니다.

**2. 데이터셋 준비 (Dataset Preparation)**
- Instruction Tuning용 데이터셋 포맷을 이해합니다.
- Chat Template을 적용하여 학습 데이터를 전처리합니다.

**3. LoRA Fine-tuning**
- LoRA 설정을 구성하고 학습을 실행합니다.

**4. 모델 평가 및 저장 (Evaluation & Saving)**
- Fine-tuning 전/후 성능을 비교합니다.
- LoRA Adapter를 저장하고 다시 로드하는 방법을 학습합니다.

**평가 방법**
1. 각 문제의 조건(Condition)을 모두 만족해야 합니다.
2. 제공된 테스트 코드가 에러 없이 실행되고, 예상된 결과가 출력되어야 합니다.
3. Fine-tuning 후 모델이 학습 데이터의 스타일을 따라가는지 확인합니다.

### Part 1. SLM과 Fine-tuning 이론

**SLM vs LLM 비교**

| 구분 | SLM | LLM |
|------|-----|-----|
| 파라미터 | 1B ~ 8B | 70B ~ 400B+ |
| 예시 모델 | Phi-3, Gemma-2, Qwen3 | GPT-4, Claude, Llama-70B |
| 학습 비용 | GPU 1장으로 가능 | 수십~수백 GPU 필요 |
| 배포 | 온디바이스 가능 | 클라우드 필수 |

**LoRA (Low-Rank Adaptation)**

- **LoRA**: 전체 파라미터 대신 저차원 행렬만 학습 (0.1~1%만 학습)
- 기존 모델 가중치는 동결(freeze)하고, 작은 어댑터 행렬만 학습
- 메모리 효율적이며 빠른 학습 가능

*메모리 사용량 비교 (4B 모델 기준): Full FT ~24GB → LoRA ~10GB*

### Part 2. 실습 환경 설정

In [ ]:
# [필수] GPU 확인 - T4 GPU가 할당되었는지 확인
!nvidia-smi

In [ ]:
# [필수] 라이브러리 설치 (실행 후 런타임 재시작이 필요할 수 있습니다)
!pip install -q transformers accelerate peft trl datasets huggingface_hub

In [ ]:
# 라이브러리 로드 및 버전 확인
import transformers
import peft
import trl
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
print(f"Transformers: {transformers.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"TRL: {trl.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# 사용할 모델 ID (Qwen3 4B - 다국어 지원 및 T4에서 학습 가능)
MODEL_ID = "Qwen/Qwen3-4B-Instruct-2507"

### Q1. 베이스 모델을 로드하고 추론 함수를 작성하세요.

> 조건1: `AutoModelForCausalLM`과 `AutoTokenizer`를 사용하여 모델과 토크나이저를 로드할 것

> 조건2: `torch.float16` 정밀도와 `device_map="auto"`를 사용할 것

> 조건3: Chat Template을 적용하여 응답을 생성하는 함수를 작성할 것

> 조건4: 시스템 프롬프트에 "고객 서비스 상담원" 역할을 부여할 것

In [ ]:
# A1.

# 조건1, 조건2: 모델과 토크나이저 로드 (float16 + device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"모델 로드 완료: {MODEL_ID}")
print(f"모델 파라미터 수: {base_model.num_parameters():,}")

# 조건3, 조건4: Chat Template을 적용한 추론 함수 (시스템 프롬프트: 고객 서비스 상담원)
def generate_response(question, model=base_model, max_new_tokens=256):
    messages = [
        {"role": "system", "content": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요."},
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return response

# 테스트
test_response = generate_response("안녕하세요?")
print(f"테스트 응답: {test_response}")

### Q2. Fine-tuning 전 베이스 모델의 성능을 테스트하세요.

> 조건1: `test_questions` 리스트의 각 질문에 대해 `generate_response()` 함수를 호출하여 응답을 생성할 것

> 조건2: 각 질문과 응답을 `base_responses` 딕셔너리에 `{질문: 응답}` 형태로 저장할 것

> 조건3: `for` 루프 내에서 각 질문과 응답을 `print()`로 출력할 것

> 조건4: 출력 형식은 `질문: {question}` 과 `답변: {response}` 형태로 할 것

In [ ]:
# 테스트 질문들 (Fine-tuning 후 비교용)
test_questions = [
    "배송이 언제 오나요?",
    "환불은 어떻게 하나요?",
    "제품이 불량이에요. 어떻게 해야 하나요?",
    "회원 탈퇴는 어떻게 하나요?",
    "적립금은 어떻게 사용하나요?"
]

In [ ]:
# A2.

# 조건2: base_responses 딕셔너리 생성
base_responses = {}

# 조건1, 조건3, 조건4: 각 질문에 대해 응답 생성 및 출력
for question in test_questions:
    response = generate_response(question)
    base_responses[question] = response
    print(f"질문: {question}")
    print(f"답변: {response}")
    print("-" * 50)

In [ ]:
# 메모리 정리 (LoRA 학습을 위해)
del base_model
torch.cuda.empty_cache()
import gc
gc.collect()
print("메모리 정리 완료!")

### Part 3. 데이터셋 준비

**Instruction 데이터셋 포맷**
```json
{
    "instruction": "시스템 프롬프트 (역할 정의)",
    "input": "사용자 질문",
    "output": "AI 답변"
}
```

*핵심: 데이터 품질 > 데이터 양 (50~100개의 고품질 데이터가 1000개의 저품질보다 효과적)*

In [ ]:
# 고객 서비스 학습 데이터
# 실제로는 더 많은 데이터가 필요합니다 (최소 50개 이상 권장)
training_data = [
    # 배송 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "배송이 언제 오나요?",
        "output": "안녕하세요, 고객님! 주문하신 상품은 결제 완료 후 영업일 기준 2~3일 내에 배송됩니다. 정확한 배송 현황은 마이페이지 > 주문내역에서 운송장 번호로 조회하실 수 있습니다. 추가 문의사항이 있으시면 말씀해 주세요!"
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "배송 조회는 어디서 하나요?",
        "output": "배송 조회는 마이페이지 > 주문내역에서 가능합니다. 해당 주문의 '배송조회' 버튼을 클릭하시면 택배사 배송 추적 페이지로 이동합니다. 운송장 번호가 아직 등록되지 않았다면 발송 준비 중이니 조금만 기다려 주세요!"
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "해외 배송도 되나요?",
        "output": "네, 해외 배송 가능합니다! 해외 배송은 국가별로 배송비와 소요 기간이 다르며, 주문 시 배송지 국가를 선택하시면 예상 배송비를 확인하실 수 있습니다. 보통 아시아 지역은 7~10일, 미주/유럽은 2~3주 정도 소요됩니다."
    },
    # 환불 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "환불은 어떻게 하나요?",
        "output": "환불 절차를 안내해 드리겠습니다. 마이페이지 > 주문내역에서 해당 주문의 '환불 신청' 버튼을 클릭해 주세요. 상품 수령 후 7일 이내, 미개봉 상태라면 100% 환불이 가능합니다. 환불 처리는 신청 후 영업일 기준 3~5일 내에 완료됩니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "환불 처리 기간은 얼마나 걸리나요?",
        "output": "환불 신청이 승인되면 결제 수단에 따라 처리 기간이 다릅니다. 신용카드는 영업일 기준 3~5일, 계좌이체는 1~2일, 포인트 결제는 즉시 복원됩니다. 환불 진행 상황은 마이페이지에서 확인하실 수 있습니다."
    },
    # 교환/불량 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "제품이 불량이에요. 어떻게 해야 하나요?",
        "output": "불편을 드려 정말 죄송합니다. 불량 제품은 무료로 교환 또는 환불해 드립니다. 불량 부분이 보이도록 사진을 찍어 고객센터(1234-5678)로 보내주시면, 확인 후 즉시 새 제품을 발송해 드리겠습니다. 반품 택배비도 저희가 부담합니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "교환 신청은 어떻게 하나요?",
        "output": "교환 신청은 마이페이지 > 주문내역에서 가능합니다. 교환하실 상품을 선택하고 '교환 신청' 버튼을 클릭한 뒤, 교환 사유와 원하시는 옵션(사이즈, 색상 등)을 선택해 주세요. 단순 변심 교환 시 왕복 배송비 5,000원이 발생합니다."
    },
    # 회원 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "회원 탈퇴는 어떻게 하나요?",
        "output": "회원 탈퇴는 마이페이지 > 회원정보 > 회원탈퇴에서 진행하실 수 있습니다. 탈퇴 시 보유하신 적립금과 쿠폰은 모두 소멸되며, 주문 내역은 관련 법률에 따라 일정 기간 보관 후 삭제됩니다. 탈퇴 전 확인해 주세요!"
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "비밀번호를 잊어버렸어요.",
        "output": "비밀번호 분실 시 로그인 페이지의 '비밀번호 찾기'를 클릭해 주세요. 가입하신 이메일로 비밀번호 재설정 링크가 발송됩니다. 메일이 오지 않으면 스팸함을 확인하시거나, 고객센터로 연락 주시면 도와드리겠습니다."
    },
    # 적립금/쿠폰 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "적립금은 어떻게 사용하나요?",
        "output": "적립금은 주문 결제 시 현금처럼 사용하실 수 있습니다. 결제 페이지에서 '적립금 사용' 항목에 사용하실 금액을 입력해 주세요. 단, 최소 주문금액 10,000원 이상일 때 사용 가능하며, 적립금은 최대 결제금액의 10%까지 사용 가능합니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "쿠폰 등록은 어디서 하나요?",
        "output": "쿠폰 등록은 마이페이지 > 쿠폰함에서 하실 수 있습니다. '쿠폰 등록' 버튼을 클릭하고 쿠폰 코드를 입력해 주세요. 등록된 쿠폰은 결제 시 자동으로 적용 가능한 목록에 표시됩니다. 쿠폰마다 사용 조건이 다르니 유효기간도 확인해 주세요!"
    },
    # 주문 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "주문 취소는 어떻게 하나요?",
        "output": "주문 취소는 '배송 준비 중' 상태 전까지 가능합니다. 마이페이지 > 주문내역에서 해당 주문의 '주문 취소' 버튼을 클릭해 주세요. 이미 발송된 경우에는 수령 후 반품 절차를 진행해 주시면 됩니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "주문 내역은 어디서 확인하나요?",
        "output": "주문 내역은 마이페이지 > 주문내역에서 확인하실 수 있습니다. 주문 날짜, 상품 정보, 결제 금액, 배송 상태를 한눈에 보실 수 있습니다. 비회원 주문의 경우 '비회원 주문조회'에서 주문번호와 연락처로 조회 가능합니다."
    },
    # 결제 관련
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "어떤 결제 수단을 사용할 수 있나요?",
        "output": "다양한 결제 수단을 지원합니다! 신용카드, 체크카드, 계좌이체, 무통장입금, 카카오페이, 네이버페이, 토스 등 간편결제도 모두 사용 가능합니다. 무통장입금은 주문 후 24시간 내에 입금해 주셔야 자동 취소되지 않습니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "카드 할부 가능한가요?",
        "output": "네, 신용카드 할부 결제 가능합니다! 5만원 이상 결제 시 2~12개월 할부를 선택하실 수 있습니다. 무이자 할부 행사는 카드사별로 다르니, 결제 페이지에서 해당 카드사의 무이자 혜택을 확인해 주세요."
    },
    # 기타
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "고객센터 운영 시간이 어떻게 되나요?",
        "output": "고객센터는 평일 오전 9시부터 오후 6시까지 운영됩니다. 점심시간(12시~1시)에는 상담이 어려울 수 있습니다. 주말 및 공휴일은 휴무이며, 1:1 문의 게시판은 24시간 접수 가능합니다. 전화번호는 1234-5678입니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "영수증 발급 가능한가요?",
        "output": "네, 영수증 발급 가능합니다! 마이페이지 > 주문내역에서 해당 주문의 '영수증 출력' 버튼을 클릭하시면 됩니다. 현금영수증은 결제 시 등록하신 번호로 자동 발급되며, 세금계산서가 필요하시면 고객센터로 사업자등록증을 보내주세요."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "재입고 알림 받을 수 있나요?",
        "output": "네, 품절 상품의 재입고 알림을 받으실 수 있습니다! 해당 상품 페이지에서 '재입고 알림 신청' 버튼을 클릭하고 연락처를 등록해 주세요. 재입고 시 SMS 또는 카카오톡으로 알림을 보내드립니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "포장 선물 가능한가요?",
        "output": "네, 선물 포장 서비스를 제공하고 있습니다! 주문 시 '선물 포장' 옵션을 선택해 주시면 예쁜 선물 박스와 리본으로 포장해 드립니다. 포장비는 2,000원이며, 메시지 카드도 무료로 동봉 가능합니다."
    },
    # 추가 데이터
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "배송비는 얼마인가요?",
        "output": "기본 배송비는 3,000원이며, 3만원 이상 구매 시 무료 배송입니다! 도서산간 지역은 추가 배송비 3,000원이 발생할 수 있습니다. 무거운 대형 상품은 별도 배송비가 적용되며, 상품 페이지에서 확인하실 수 있습니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "상품 문의는 어디서 하나요?",
        "output": "상품 문의는 해당 상품 페이지 하단의 'Q&A' 게시판에서 하실 수 있습니다. 상품 상세 정보, 사이즈, 재질 등 궁금한 점을 남겨주시면 담당자가 빠르게 답변드립니다. 개인정보가 포함된 문의는 1:1 문의를 이용해 주세요."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "주문 수정이 가능한가요?",
        "output": "주문 수정은 '결제 완료' 상태일 때만 가능합니다. 마이페이지 > 주문내역에서 배송지 변경이나 옵션 변경을 하실 수 있습니다. 이미 '배송 준비 중'으로 변경되었다면 주문 취소 후 재주문 부탁드립니다."
    },
    {
        "instruction": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요.",
        "input": "앱에서도 주문할 수 있나요?",
        "output": "네, 모바일 앱에서도 편리하게 주문하실 수 있습니다! 앱스토어 또는 구글플레이에서 저희 앱을 다운로드해 주세요. 앱 전용 할인 쿠폰과 푸시 알림 혜택도 받으실 수 있습니다."
    }
]

### Q3. 학습 데이터를 Chat 템플릿 형식으로 변환하세요.

> 조건1: `formatting_func()` 함수 내에서 `messages` 리스트를 생성할 것

> 조건2: `messages`는 `{"role": "system", "content": instruction}`, `{"role": "user", "content": input}`, `{"role": "assistant", "content": output}` 세 개의 딕셔너리로 구성할 것

> 조건3: `tokenizer.apply_chat_template(messages, tokenize=False)`를 사용하여 포맷팅할 것

> 조건4: 함수는 `{"text": formatted_text}` 형태의 딕셔너리를 반환할 것

In [ ]:
# A3.

def formatting_func(example):
    # 조건1: messages 리스트 생성
    # 조건2: system, user, assistant 세 개의 딕셔너리로 구성
    messages = [
        {"role": "system", "content": example["instruction"]},
        {"role": "user", "content": example["input"]},
        {"role": "assistant", "content": example["output"]}
    ]
    # 조건3: apply_chat_template 사용
    formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
    # 조건4: {"text": formatted_text} 형태로 반환
    return {"text": formatted_text}

# 데이터셋 생성 및 변환
dataset = Dataset.from_list(training_data)
dataset = dataset.map(formatting_func)

print(f"데이터셋 크기: {len(dataset)}")
print(f"컬럼: {dataset.column_names}")
print(f"\n첫 번째 샘플 (text 필드):\n{dataset[0]['text'][:500]}...")

### Q4. 데이터셋의 토큰 길이를 분석하세요.

> 조건1: `count_tokens()` 함수 내에서 `tokenizer(example["text"])`를 사용하여 토큰화할 것

> 조건2: 토큰화 결과의 `input_ids` 길이를 `len()`으로 계산할 것

> 조건3: 함수는 `{"token_count": 토큰_길이}` 형태의 딕셔너리를 반환할 것

> 조건4: 토큰 길이 분석 결과(평균, 최대, 최소)가 정상적으로 출력되어야 할 것

In [ ]:
# A4.

def count_tokens(example):
    # 조건1: tokenizer(example["text"])로 토큰화
    tokenized = tokenizer(example["text"])
    # 조건2: input_ids 길이를 len()으로 계산
    token_length = len(tokenized["input_ids"])
    # 조건3: {"token_count": 토큰_길이} 형태로 반환
    return {"token_count": token_length}

# 토큰 길이 분석
dataset_with_tokens = dataset.map(count_tokens)
token_counts = dataset_with_tokens["token_count"]

# 조건4: 평균, 최대, 최소 출력
print(f"토큰 길이 분석 결과:")
print(f"  평균: {sum(token_counts) / len(token_counts):.1f}")
print(f"  최대: {max(token_counts)}")
print(f"  최소: {min(token_counts)}")
print(f"  총 샘플 수: {len(token_counts)}")

### Part 4. LoRA Fine-tuning

### Q5. LoRA Fine-tuning을 위한 모델을 로드하세요.

> 조건1: `AutoModelForCausalLM.from_pretrained()`를 사용하여 모델을 로드할 것

> 조건2: `torch_dtype=torch.float16`으로 16비트 정밀도를 설정할 것

> 조건3: `device_map="auto"`로 자동 디바이스 매핑을 설정할 것

> 조건4: 모델 로드 후 메모리 사용량이 출력되어야 할 것

In [ ]:
# A5.

# 조건1: AutoModelForCausalLM.from_pretrained()로 모델 로드
# 조건2: torch_dtype=torch.float16
# 조건3: device_map="auto"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

print(f"모델 로드 완료: {MODEL_ID}")
print(f"모델 파라미터 수: {model.num_parameters():,}")

# 조건4: 메모리 사용량 출력
if torch.cuda.is_available():
    print(f"GPU 메모리 사용량: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"GPU 메모리 예약량: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

### Q6. LoRA 설정을 구성하고 모델에 적용하세요.

> 조건1: `LoraConfig()`를 사용하여 설정 객체를 생성할 것

> 조건2: `r=16`, `lora_alpha=32`, `lora_dropout=0.05`로 설정할 것

> 조건3: `target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]`로 Attention 레이어를 지정할 것

> 조건4: `task_type="CAUSAL_LM"`으로 태스크 타입을 지정할 것

In [ ]:
# A6.

from peft import LoraConfig, get_peft_model

# 조건1: LoraConfig() 설정 객체 생성
# 조건2: r=16, lora_alpha=32, lora_dropout=0.05
# 조건3: target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
# 조건4: task_type="CAUSAL_LM"
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    bias="none"
)

# LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 확인
model.print_trainable_parameters()

print(f"\nLoRA 설정:")
print(f"  r (rank): {lora_config.r}")
print(f"  lora_alpha: {lora_config.lora_alpha}")
print(f"  lora_dropout: {lora_config.lora_dropout}")
print(f"  target_modules: {lora_config.target_modules}")
print(f"  task_type: {lora_config.task_type}")

### Q7. 학습 설정을 구성하고 SFTTrainer를 초기화하세요.

> 조건1: `TrainingArguments()`에서 `num_train_epochs=3`으로 설정할 것

> 조건2: `per_device_train_batch_size=2`로 배치 크기를 설정할 것

> 조건3: `gradient_accumulation_steps=4`로 그래디언트 누적 스텝을 설정할 것

> 조건4: 설정 완료 후 실효 배치 크기(per_device_train_batch_size × gradient_accumulation_steps = 8)가 출력되어야 할 것

In [ ]:
# A7.

from trl import SFTTrainer, SFTConfig

# 조건1: num_train_epochs=3
# 조건2: per_device_train_batch_size=2
# 조건3: gradient_accumulation_steps=4
training_args = SFTConfig(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=10,
    logging_steps=5,
    save_strategy="epoch",
    fp16=True,
    optim="adamw_torch",
    report_to="none",
    max_length=512
)

# SFTTrainer 초기화
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer
)

# 조건4: 실효 배치 크기 출력
effective_batch_size = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
print(f"학습 설정:")
print(f"  에폭 수: {training_args.num_train_epochs}")
print(f"  배치 크기: {training_args.per_device_train_batch_size}")
print(f"  그래디언트 누적 스텝: {training_args.gradient_accumulation_steps}")
print(f"  실효 배치 크기: {training_args.per_device_train_batch_size} × {training_args.gradient_accumulation_steps} = {effective_batch_size}")
print(f"  학습률: {training_args.learning_rate}")

### Q8. 학습을 실행하고 결과를 확인하세요.

> 조건1: `trainer.train()`을 호출하여 학습을 실행할 것

> 조건2: 학습 결과를 `train_result` 변수에 저장할 것

> 조건3: `train_result.metrics`에서 `train_runtime`, `train_loss`, `train_samples_per_second`를 출력할 것

> 조건4: 학습 중 Loss가 점차 감소하는지 로그를 확인할 것

In [ ]:
# A8.

# 조건1: trainer.train() 호출
# 조건2: train_result 변수에 저장
train_result = trainer.train()

# 조건3: train_runtime, train_loss, train_samples_per_second 출력
metrics = train_result.metrics
print(f"\n학습 결과:")
print(f"  학습 시간: {metrics['train_runtime']:.2f}초")
print(f"  최종 Loss: {metrics['train_loss']:.4f}")
print(f"  초당 샘플 수: {metrics['train_samples_per_second']:.2f}")

# 조건4: Loss 감소 확인은 위 학습 로그에서 확인 가능
print(f"\n학습 로그에서 Loss가 점차 감소하는지 확인하세요.")

### Part 5. 모델 평가 및 저장

### Q9. Fine-tuning 전/후 성능을 비교하세요.

> 조건1: `generate_response_finetuned()` 함수를 작성하여 Fine-tuned 모델로 응답을 생성할 것

> 조건2: 함수 내에서 Q1과 동일한 Chat Template 방식(`messages` 리스트 + `apply_chat_template()`)을 사용할 것

> 조건3: `model.generate()`를 호출하고 `tokenizer.decode()`로 응답 텍스트를 추출할 것

> 조건4: `test_questions`의 각 질문에 대해 Fine-tuning 전(`base_responses`)과 후(`finetuned_responses`) 응답을 나란히 출력하여 비교할 것

In [ ]:
# A9.

# 조건1: Fine-tuned 모델로 응답 생성 함수
def generate_response_finetuned(question, max_new_tokens=256):
    # 조건2: Q1과 동일한 Chat Template 방식
    messages = [
        {"role": "system", "content": "당신은 친절한 고객 서비스 상담원입니다. 고객의 질문에 정확하고 도움이 되는 답변을 제공하세요."},
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    # 조건3: model.generate() 호출 및 tokenizer.decode()로 응답 추출
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return response

# 조건4: Fine-tuning 전/후 비교
finetuned_responses = {}

for question in test_questions:
    response = generate_response_finetuned(question)
    finetuned_responses[question] = response
    print(f"질문: {question}")
    print(f"[Fine-tuning 전] 답변: {base_responses[question]}")
    print(f"[Fine-tuning 후] 답변: {response}")
    print("=" * 70)

### Q10. LoRA Adapter를 저장하고 다시 로드하는 코드를 작성하세요.

> 조건1: `trainer.save_model(ADAPTER_PATH)`를 사용하여 LoRA Adapter를 저장할 것

> 조건2: `tokenizer.save_pretrained(ADAPTER_PATH)`를 사용하여 토크나이저도 함께 저장할 것

> 조건3: `load_finetuned_model()` 함수에서 `AutoModelForCausalLM.from_pretrained()`로 베이스 모델을 로드한 후, `PeftModel.from_pretrained()`로 Adapter를 적용할 것

> 조건4: 저장된 파일 목록과 각 파일의 용량(MB)을 `os.listdir()`과 `os.path.getsize()`로 출력할 것

In [ ]:
# A10.

import os
from peft import PeftModel

ADAPTER_PATH = "./lora_adapter"

# 조건1: LoRA Adapter 저장
trainer.save_model(ADAPTER_PATH)

# 조건2: 토크나이저 저장
tokenizer.save_pretrained(ADAPTER_PATH)

# 조건4: 저장된 파일 목록과 용량 출력
print(f"저장 경로: {ADAPTER_PATH}")
print(f"\n저장된 파일 목록:")
for filename in os.listdir(ADAPTER_PATH):
    filepath = os.path.join(ADAPTER_PATH, filename)
    size_mb = os.path.getsize(filepath) / (1024 * 1024)
    print(f"  {filename}: {size_mb:.2f} MB")

# 조건3: 저장된 Adapter 로드 함수
def load_finetuned_model(base_model_id, adapter_path):
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    # PeftModel.from_pretrained()로 Adapter 적용
    model = PeftModel.from_pretrained(base_model, adapter_path)
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    print(f"모델 로드 완료: {base_model_id} + {adapter_path}")
    return model, tokenizer

# 로드 테스트
print(f"\n로드 함수 테스트:")
loaded_model, loaded_tokenizer = load_finetuned_model(MODEL_ID, ADAPTER_PATH)
print("LoRA Adapter 로드 성공!")